# 🎭 FESTA Framework Demo

Demo for **FESTA (Framework for Evaluating Semantic and Temporal Assumptions)** using BLINK and VSR datasets.

**Repository**: [https://github.com/iiscleap/mllm-uncertainty-estimation](https://github.com/iiscleap/mllm-uncertainty-estimation)

In [ ]:
# Install packages
!pip install -q transformers==4.36.2 torch torchvision pillow accelerate requests matplotlib seaborn numpy pandas

In [ ]:
# Download dataset examples
import os, requests, json
os.makedirs('examples', exist_ok=True)

base_url = "https://raw.githubusercontent.com/iiscleap/mllm-uncertainty-estimation/main/examples/"
example_images = [
    "val_Spatial_Relation_1.jpg", "val_Spatial_Relation_1_blur1.jpg",
    "val_Spatial_Relation_10.jpg", "val_Spatial_Relation_10_contrast1.jpg",
    "val_Spatial_Relation_25.jpg", "val_Spatial_Relation_25_noise1.jpg",
    "val_Spatial_Relation_50.jpg", "val_Spatial_Relation_50_bw1.jpg",
    "val_Spatial_Relation_75.jpg", "val_Spatial_Relation_75_masking1.jpg",
    "val_Spatial_Reasoning_111.jpg", "val_Spatial_Reasoning_125.jpg"
]

print("📥 Downloading examples...")
for img_name in example_images:
    try:
        response = requests.get(base_url + img_name)
        if response.status_code == 200:
            with open(f'examples/{img_name}', 'wb') as f:
                f.write(response.content)
            print(f"✅ {img_name}")
    except Exception as e:
        print(f"❌ {img_name}: {e}")

print("✅ Ready!")

In [ ]:
import torch, matplotlib.pyplot as plt, seaborn as sns
from PIL import Image
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

print(f"🔥 CUDA: {torch.cuda.is_available()}")

# Load demo config with real CSV data
demo_config = {
    "equivalent_examples": [
        {
            "id": "blink_val_Spatial_Relation_1_blur1",
            "dataset": "BLINK",
            "original_image": "val_Spatial_Relation_1.jpg",
            "perturbed_image": "val_Spatial_Relation_1_blur1.jpg",
            "original_question": "Is the cat standing above a car?",
            "perturbed_question": "How are the cat and the car situated relative to each other?",
            "expected_answer": "B - The cat is not above the car",
            "expected_failure": False,
            "perturbation_type": "blur1"
        },
        {
            "id": "blink_val_Spatial_Relation_10_contrast1",
            "dataset": "BLINK",
            "original_image": "val_Spatial_Relation_10.jpg",
            "perturbed_image": "val_Spatial_Relation_10_contrast1.jpg",
            "original_question": "Is the person in contact with the laptop?",
            "perturbed_question": "What's the relative placement of the person and the laptop?",
            "expected_answer": "A - The person is in contact with the laptop",
            "expected_failure": False,
            "perturbation_type": "contrast1"
        },
        {
            "id": "blink_val_Spatial_Relation_25_noise1",
            "dataset": "BLINK",
            "original_image": "val_Spatial_Relation_25.jpg",
            "perturbed_image": "val_Spatial_Relation_25_noise1.jpg",
            "original_question": "Is the sheep situated outside the bottle?",
            "perturbed_question": "How are the sheep and the bottle situated relative to each other?",
            "expected_answer": "A - The sheep is outside the bottle",
            "expected_failure": False,
            "perturbation_type": "noise1"
        },
        {
            "id": "blink_val_Spatial_Relation_50_bw1",
            "dataset": "BLINK",
            "original_image": "val_Spatial_Relation_50.jpg",
            "perturbed_image": "val_Spatial_Relation_50_bw1.jpg",
            "original_question": "Is the sandwich positioned to the left of the laptop?",
            "perturbed_question": "What's the relative placement of the sandwich and the laptop?",
            "expected_answer": "B - The sandwich is not to the left of the laptop",
            "expected_failure": False,
            "perturbation_type": "bw1"
        },
        {
            "id": "blink_val_Spatial_Relation_75_masking1",
            "dataset": "BLINK",
            "original_image": "val_Spatial_Relation_75.jpg",
            "perturbed_image": "val_Spatial_Relation_75_masking1.jpg",
            "original_question": "Is there a person in physical contact with the zebra?",
            "perturbed_question": "What is the spatial arrangement of the person and the zebra?",
            "expected_answer": "B - The person is not in physical contact with the zebra",
            "expected_failure": True,
            "perturbation_type": "masking1"
        }
    ],
    "complementary_examples": [
        {
            "id": "blink_val_Spatial_Relation_1_negated",
            "dataset": "BLINK",
            "image": "val_Spatial_Relation_1.jpg",
            "original_question": "Is the cat standing above a car?",
            "complementary_question": "Is the car positioned ahead of the cat?",
            "expected_original_answer": "B - The cat is not above the car",
            "expected_complementary_answer": "B - The car is not ahead of the cat",
            "expected_failure": False
        },
        {
            "id": "blink_val_Spatial_Relation_10_negated",
            "dataset": "BLINK",
            "image": "val_Spatial_Relation_10.jpg",
            "original_question": "Is the person in contact with the laptop?",
            "complementary_question": "Is the laptop located at a distance from the individual?",
            "expected_original_answer": "A - The person is in contact with the laptop",
            "expected_complementary_answer": "A - The laptop is at a distance from the individual",
            "expected_failure": False
        },
        {
            "id": "vsr_val_Spatial_Reasoning_111",
            "dataset": "VSR",
            "image": "val_Spatial_Reasoning_111.jpg",
            "original_question": "The dog has the motorcycle positioned behind it.",
            "complementary_question": "The dog is behind the motorcycle.",
            "expected_original_answer": "A - The dog is behind the motorcycle",
            "expected_complementary_answer": "A - The dog is behind the motorcycle",
            "expected_failure": False
        },
        {
            "id": "vsr_val_Spatial_Reasoning_125",
            "dataset": "VSR",
            "image": "val_Spatial_Reasoning_125.jpg",
            "original_question": "The horse is positioned above the chair.",
            "complementary_question": "The horse has the chair positioned behind it.",
            "expected_original_answer": "B - The horse is not above the chair",
            "expected_complementary_answer": "B - The horse is not behind the chair",
            "expected_failure": True
        }
    ]
}

print("📋 Config loaded!")

In [ ]:
# Load LLaVA model
print("📥 Loading model...")

model_name = "llava-hf/llava-v1.6-mistral-7b-hf"
processor = LlavaNextProcessor.from_pretrained(model_name)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_name, torch_dtype=torch.float16, device_map="auto", load_in_8bit=True
)

print("✅ Model loaded!")

In [ ]:
# Helper functions
def get_model_response(image, question, max_new_tokens=50):
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    inputs = processor(prompt, image, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs, max_new_tokens=max_new_tokens, do_sample=False,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    
    response = processor.decode(output[0], skip_special_tokens=True)
    return response.split("ASSISTANT:")[-1].strip()

def load_example_image(image_name):
    try:
        return Image.open(f'examples/{image_name}')
    except:
        print(f"⚠️ Could not load {image_name}")
        return Image.new('RGB', (224, 224), color='lightgray')

def display_example(orig_img, pert_img, orig_q, pert_q, orig_resp, pert_resp, is_failure, desc):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(orig_img)
    axes[0].set_title("Original", fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(pert_img)
    axes[1].set_title("Perturbed", fontweight='bold')
    axes[1].axis('off')
    
    status = "❌ FAILURE" if is_failure else "✅ SUCCESS"
    color = "red" if is_failure else "green"
    plt.suptitle(f"EQUIVALENT - {status}", fontsize=14, fontweight='bold', color=color)
    plt.tight_layout()
    plt.show()
    
    print(f"📝 Original Q: {orig_q}")
    print(f"🤖 Original R: {orig_resp}")
    print(f"📝 Perturbed Q: {pert_q}")
    print(f"🤖 Perturbed R: {pert_resp}\n")

def display_complementary_example(img, orig_q, comp_q, orig_resp, comp_resp, is_failure, desc):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(img)
    ax.axis('off')
    
    status = "❌ FAILURE" if is_failure else "✅ SUCCESS"
    color = "red" if is_failure else "green"
    plt.suptitle(f"COMPLEMENTARY - {status}", fontsize=14, fontweight='bold', color=color)
    plt.show()
    
    print(f"📝 Original Q: {orig_q}")
    print(f"🤖 Original R: {orig_resp}")
    print(f"📝 Complementary Q: {comp_q}")
    print(f"🤖 Complementary R: {comp_resp}\n")

print("🛠️ Functions ready!")

In [ ]:
# Run equivalent examples
equivalent_results = []

for i, example in enumerate(demo_config["equivalent_examples"], 1):
    print(f"🧪 EQUIVALENT EXAMPLE {i}: {example['dataset']} - {example['perturbation_type']}")
    
    orig_img = load_example_image(example['original_image'])
    pert_img = load_example_image(example['perturbed_image'])
    
    orig_resp = get_model_response(orig_img, example['original_question'])
    pert_resp = get_model_response(pert_img, example['perturbed_question'])
    
    is_failure = example['expected_failure']
    
    display_example(orig_img, pert_img, example['original_question'], 
                   example['perturbed_question'], orig_resp, pert_resp,
                   is_failure, example['perturbation_type'])
    
    equivalent_results.append({
        'example_id': example['id'],
        'is_failure': is_failure,
        'original_response': orig_resp,
        'perturbed_response': pert_resp
    })

In [ ]:
# Run complementary examples
complementary_results = []

for i, example in enumerate(demo_config["complementary_examples"], 1):
    print(f"🧪 COMPLEMENTARY EXAMPLE {i}: {example['dataset']}")
    
    img = load_example_image(example['image'])
    
    orig_resp = get_model_response(img, example['original_question'])
    comp_resp = get_model_response(img, example['complementary_question'])
    
    is_failure = example['expected_failure']
    
    display_complementary_example(img, example['original_question'],
                                 example['complementary_question'],
                                 orig_resp, comp_resp, is_failure, "")
    
    complementary_results.append({
        'example_id': example['id'],
        'is_failure': is_failure,
        'original_response': orig_resp,
        'complementary_response': comp_resp
    })

In [ ]:
# Summary visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

equiv_success = [0 if result['is_failure'] else 1 for result in equivalent_results]
comp_success = [0 if result['is_failure'] else 1 for result in complementary_results]

# Equivalent summary
equiv_labels = ['Blur', 'Contrast', 'Noise', 'B&W', 'Masking']
equiv_colors = ['green' if x == 1 else 'red' for x in equiv_success]

bars1 = ax1.bar(equiv_labels, [1]*5, color=equiv_colors, alpha=0.7)
ax1.set_title('EQUIVALENT Examples', fontweight='bold')
ax1.set_ylabel('Success Rate')
ax1.set_ylim(0, 1.2)

for bar, success in zip(bars1, equiv_success):
    status = '✅' if success else '❌'
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
            status, ha='center', fontweight='bold', fontsize=16)

# Complementary summary
comp_labels = ['BLINK\nSpatial 1', 'BLINK\nSpatial 10', 'VSR\nDog-Bike', 'VSR\nHorse-Chair']
comp_colors = ['blue' if x == 1 else 'red' for x in comp_success]

bars2 = ax2.bar(comp_labels, [1]*4, color=comp_colors, alpha=0.7)
ax2.set_title('COMPLEMENTARY Examples', fontweight='bold')
ax2.set_ylabel('Success Rate')
ax2.set_ylim(0, 1.2)

for bar, success in zip(bars2, comp_success):
    status = '✅' if success else '❌'
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
            status, ha='center', fontweight='bold', fontsize=16)

plt.tight_layout()
plt.suptitle('🎭 FESTA Framework Results', fontsize=16, fontweight='bold', y=1.02)
plt.show()

# Stats
total_equiv = sum(equiv_success)
total_comp = sum(comp_success)
overall = total_equiv + total_comp

print("\n" + "="*50)
print("📈 FESTA RESULTS")
print("="*50)
print(f"🔄 Equivalent: {total_equiv}/5 ({total_equiv/5*100:.0f}%)")
print(f"🔄 Complementary: {total_comp}/4 ({total_comp/4*100:.0f}%)")
print(f"📊 Overall: {overall}/9 ({overall/9*100:.0f}%)")
print("="*50)